# unstack_normalize_dfs_xgb_classifier

In [1]:
from greenguard.demo import load_demo

target_times, readings = load_demo()

Using TensorFlow backend.


In [2]:
pipeline_name = 'unstack_normalize_dfs_xgb_classifier'

In [3]:
from greenguard.pipeline import GreenGuardPipeline

pipeline = GreenGuardPipeline(pipeline_name)

In [4]:
pipeline.template['primitives']

['pandas.DataFrame.resample',
 'pandas.DataFrame.unstack',
 'featuretools.EntitySet.entity_from_dataframe',
 'featuretools.EntitySet.normalize_entity',
 'featuretools.dfs',
 'mlprimitives.custom.feature_extraction.CategoricalEncoder',
 'xgboost.XGBClassifier']

# Step by Step execution

## Input Data

In [5]:
readings.head()

,turbine_id,signal_id,timestamp,value
0,T001,S01,2013-01-10,323.0
1,T001,S02,2013-01-10,320.0
2,T001,S03,2013-01-10,284.0
3,T001,S04,2013-01-10,348.0
4,T001,S05,2013-01-10,273.0


In [6]:
target_times.head()

,turbine_id,cutoff_time,target
0,T001,2013-01-12,0
1,T001,2013-01-13,0
2,T001,2013-01-14,0
3,T001,2013-01-15,1
4,T001,2013-01-16,0


## Data Preparation (part of GreenGuard Pipeline)

* Input: target_times, readings, turbines
* Output: X, y, readings, turbines
* Effect: target_times has been split into X and y

## pandas.DataFrame.resample

* Input: readings
* Output: readings (resampled)
* Effect: readings have been resampled to the indicated resample rule and turbine_id,
  signal_id and timestamp have been set as a multi-index

In [7]:
step = 0
context = pipeline.fit(target_times, readings, output_=step)

In [8]:
context.keys()

dict_keys(['readings', 'turbines', 'X', 'y'])

In [9]:
context['readings'].head()

value
turbine_id signal_id timestamp                 
T001       S01       2013-01-10 00:00:00  323.0
                     2013-01-10 00:10:00  346.0
                     2013-01-10 00:20:00  407.0
                     2013-01-10 00:30:00  257.0
                     2013-01-10 00:40:00  267.0

## pandas.DataFrame.unstack

* Input: readings (resampled)
* Output: readings (unstacked)
* Effect: readings have been unstacked

In [10]:
step = 1
context = pipeline.fit(**context, output_=step, start_=step)

In [11]:
context.keys()

dict_keys(['readings', 'turbines', 'X', 'y'])

In [12]:
context['readings'].head()

,turbine_id,timestamp,value_S01,value_S02,value_S03,value_S04,value_S05,value_S06,value_S07,value_S08,...,value_S17,value_S18,value_S19,value_S20,value_S21,value_S22,value_S23,value_S24,value_S25,value_S26
0,T001,2013-01-10 00:00:00,323.0,320.0,284.0,348.0,273.0,342.0,280.0,3197842.0,...,11.7,3131020.0,55.0,55.0,47.0,58.0,45.0,58.0,47.0,356.0
1,T001,2013-01-10 00:10:00,346.0,384.0,367.0,411.0,331.0,360.0,249.0,3197900.0,...,10.2,3131420.0,58.0,63.0,62.0,67.0,55.0,61.0,42.0,400.0
2,T001,2013-01-10 00:20:00,407.0,363.0,407.0,393.0,275.0,335.0,270.0,3197968.0,...,9.5,3131822.0,68.0,61.0,67.0,66.0,46.0,55.0,45.0,402.0
3,T001,2013-01-10 00:30:00,257.0,307.0,315.0,361.0,317.0,354.0,271.0,3198011.0,...,10.5,3132179.0,43.0,51.0,53.0,62.0,53.0,60.0,45.0,357.0
4,T001,2013-01-10 00:40:00,267.0,309.0,314.0,355.0,262.0,246.0,212.0,3198056.0,...,9.6,3132501.0,45.0,51.0,54.0,59.0,43.0,41.0,36.0,322.0


##  featuretools.EntitySet.entity_from_dataframe

* Input: readings (resampled)
* Output: entityset
* Effect: Entityset has been generated from readings

In [13]:
step = 2
context = pipeline.fit(**context, output_=step, start_=step)

In [14]:
context.keys()

dict_keys(['readings', 'turbines', 'X', 'y', 'entityset'])

In [15]:
context['entityset']

Entityset: entityset
  Entities:
    readings [Rows: 51121, Columns: 29]
  Relationships:
    No relationships

## featuretools.EntitySet.normalize_entity

* Input: entityset
* Output: entityset with relationship (readings.turbine_id with turbines.turbine_id)
* Effect: establish relation between readings and turbines

In [16]:
step = 3
context = pipeline.fit(**context, output_=step, start_=step)

In [17]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [18]:
context['entityset']

Entityset: entityset
  Entities:
    readings [Rows: 51121, Columns: 29]
    turbines [Rows: 1, Columns: 1]
  Relationships:
    readings.turbine_id -> turbines.turbine_id

## featuretools.dfs

* Input: entityset (unstacked, no turbine_id, no timestamp)
* Output: X (has additional features)
* Effect: build features for relational dataset using DFS

In [19]:
step = 4
context = pipeline.fit(**context, output_=step, start_=step)

In [20]:
context.keys() 

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [21]:
context['X'].head()

,SUM(readings.value_S22),SUM(readings.value_S02),SUM(readings.value_S01),SUM(readings.value_S16),SUM(readings.value_S10),SUM(readings.value_S14),SUM(readings.value_S11),SUM(readings.value_S23),SUM(readings.value_S19),SUM(readings.value_S06),...,MEAN(readings.value_S07),COUNT(readings),NUM_UNIQUE(readings.WEEKDAY(timestamp)),NUM_UNIQUE(readings.YEAR(timestamp)),NUM_UNIQUE(readings.MONTH(timestamp)),NUM_UNIQUE(readings.DAY(timestamp)),MODE(readings.WEEKDAY(timestamp)),MODE(readings.YEAR(timestamp)),MODE(readings.MONTH(timestamp)),MODE(readings.DAY(timestamp))
turbine_id,,,,,,,,,,,,,,,,,,,,,
T001,3240.0,19570.0,19558.0,555.2,486911931.0,468652633.0,499808026.0,2806.0,3209.0,18287.0,...,115.372414,145,2,1,1,2,4,2013,1,11
T001,5108.0,35512.0,37965.0,719.2,487487610.0,469131137.0,500401347.0,4684.0,5159.0,33056.0,...,220.434483,145,2,1,1,2,5,2013,1,12
T001,10144.0,72763.0,73948.0,921.1,488531121.0,470044626.0,501472849.0,9218.0,9170.0,76213.0,...,539.455172,145,2,1,1,2,6,2013,1,13
T001,10912.0,81198.0,87206.0,977.2,490024295.0,471497238.0,502994331.0,10382.0,10726.0,83217.0,...,576.241379,145,2,1,1,2,0,2013,1,14
T001,10353.0,65077.0,61778.0,954.3,396521849.0,374932811.0,403671026.0,8271.0,8870.0,63304.0,...,533.121739,145,2,1,1,2,1,2013,1,15


In [22]:
# features generated (the turbine_id is set as index).
len(context['X'].columns)

165

## mlprimitives.custom.feature_extraction.CategoricalEncoder

* Input: X
* Output: X (label encoded)
* Effect: encodes categorical features using OneHotLabelEncoder

In [23]:
step = 5
context = pipeline.fit(**context, output_=step, start_=step)

In [24]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [25]:
context['X'].head()

,SUM(readings.value_S22),SUM(readings.value_S02),SUM(readings.value_S01),SUM(readings.value_S16),SUM(readings.value_S10),SUM(readings.value_S14),SUM(readings.value_S11),SUM(readings.value_S23),SUM(readings.value_S19),SUM(readings.value_S06),...,MEAN(readings.value_S07),COUNT(readings),NUM_UNIQUE(readings.WEEKDAY(timestamp)),NUM_UNIQUE(readings.YEAR(timestamp)),NUM_UNIQUE(readings.MONTH(timestamp)),NUM_UNIQUE(readings.DAY(timestamp)),MODE(readings.WEEKDAY(timestamp)),MODE(readings.YEAR(timestamp)),MODE(readings.MONTH(timestamp)),MODE(readings.DAY(timestamp))
turbine_id,,,,,,,,,,,,,,,,,,,,,
T001,3240.0,19570.0,19558.0,555.2,486911931.0,468652633.0,499808026.0,2806.0,3209.0,18287.0,...,115.372414,145,2,1,1,2,4,2013,1,11
T001,5108.0,35512.0,37965.0,719.2,487487610.0,469131137.0,500401347.0,4684.0,5159.0,33056.0,...,220.434483,145,2,1,1,2,5,2013,1,12
T001,10144.0,72763.0,73948.0,921.1,488531121.0,470044626.0,501472849.0,9218.0,9170.0,76213.0,...,539.455172,145,2,1,1,2,6,2013,1,13
T001,10912.0,81198.0,87206.0,977.2,490024295.0,471497238.0,502994331.0,10382.0,10726.0,83217.0,...,576.241379,145,2,1,1,2,0,2013,1,14
T001,10353.0,65077.0,61778.0,954.3,396521849.0,374932811.0,403671026.0,8271.0,8870.0,63304.0,...,533.121739,145,2,1,1,2,1,2013,1,15


In [26]:
context['readings'].head()

,turbine_id,timestamp,value_S01,value_S02,value_S03,value_S04,value_S05,value_S06,value_S07,value_S08,...,value_S17,value_S18,value_S19,value_S20,value_S21,value_S22,value_S23,value_S24,value_S25,value_S26
0,T001,2013-01-10 00:00:00,323.0,320.0,284.0,348.0,273.0,342.0,280.0,3197842.0,...,11.7,3131020.0,55.0,55.0,47.0,58.0,45.0,58.0,47.0,356.0
1,T001,2013-01-10 00:10:00,346.0,384.0,367.0,411.0,331.0,360.0,249.0,3197900.0,...,10.2,3131420.0,58.0,63.0,62.0,67.0,55.0,61.0,42.0,400.0
2,T001,2013-01-10 00:20:00,407.0,363.0,407.0,393.0,275.0,335.0,270.0,3197968.0,...,9.5,3131822.0,68.0,61.0,67.0,66.0,46.0,55.0,45.0,402.0
3,T001,2013-01-10 00:30:00,257.0,307.0,315.0,361.0,317.0,354.0,271.0,3198011.0,...,10.5,3132179.0,43.0,51.0,53.0,62.0,53.0,60.0,45.0,357.0
4,T001,2013-01-10 00:40:00,267.0,309.0,314.0,355.0,262.0,246.0,212.0,3198056.0,...,9.6,3132501.0,45.0,51.0,54.0,59.0,43.0,41.0,36.0,322.0


## xgboost.XGBClassifier

* Input: X (label encoded and featurized)
* Output: None
* Effect: trained model

In [27]:
step = 6
context = pipeline.fit(**context, output_=step, start_=step)

In [28]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])